# ResNet-18

In [1]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class CustomDataset(Dataset):
  def __init__(self, csv_file, transform=None):
    self.data = pd.read_csv(csv_file)
    self.transform = transform
    self.label_dict = {"notinfacted": 0, "infacted": 1}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    # get image file paths from csv file
    img_name = self.data.iloc[idx, 0]
    # get the image from path
    image = Image.open(img_name)
    # get the image label from csv file
    label = self.data.iloc[idx, 1]
    # encode the image label accordingly
    encoded_label = self.label_dict[label]
    if self.transform:
      image = self.transform(image)

    return image, encoded_label


# Define transforms for the input images
transform = transforms.Compose(
    [
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        # normalization for color images
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Load train and test datasets
train_set = CustomDataset(
    csv_file="train.csv",
    transform=transform,
)
test_set = CustomDataset(
    csv_file="test.csv",
    transform=transform,
)

In [2]:
# Print set sizes
print(len(train_set))
print(len(test_set))

1030
257


In [3]:
from torch.utils.data import Dataset, DataLoader

batch_size = 32

# Create DataLoader for train set
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [4]:
# Print loader
print(len(train_loader))

33


In [5]:
import torch
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Train ResNet-18 Model

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
# from torchsummary import summary

# Define the model
model = torchvision.models.resnet18(pretrained=True)

# Modify the last fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

C:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
import time

num_epochs = 20
total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        
        # Reshape labels for BCEWithLogitsLoss
        labels = labels.float().view(-1, 1)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        # Apply sigmoid and round to get predictions
        predicted = torch.round(torch.sigmoid(outputs))
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_set)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time
    epoch_time_minutes = epoch_time / 60
    
    train_accuracy = correct_train / total_train
    
    # Save model checkpoints for each  epoch
    torch.save(model.state_dict(), f"resnet18_model_bth{batch_size}_eph{num_epochs}_{epoch}.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {epoch_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.2%}, "
          f"Time: {epoch_time:.2f} seconds "
          f"({epoch_time_minutes:.2f} minutes)")

total_end_time = time.time()
total_time = total_end_time - total_start_time
print(f"Total training time: {total_time/60:.2f} minutes")

C:\Users\User\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20], Loss: 0.3233, Train Accuracy: 87.38%, Time: 142.05 seconds (2.37 minutes)
Epoch [2/20], Loss: 0.1973, Train Accuracy: 93.01%, Time: 136.82 seconds (2.28 minutes)
Epoch [3/20], Loss: 0.1376, Train Accuracy: 95.44%, Time: 135.88 seconds (2.26 minutes)
Epoch [4/20], Loss: 0.1729, Train Accuracy: 94.17%, Time: 135.76 seconds (2.26 minutes)
Epoch [5/20], Loss: 0.1501, Train Accuracy: 94.85%, Time: 136.33 seconds (2.27 minutes)
Epoch [6/20], Loss: 0.0487, Train Accuracy: 98.64%, Time: 135.93 seconds (2.27 minutes)
Epoch [7/20], Loss: 0.0736, Train Accuracy: 97.28%, Time: 135.19 seconds (2.25 minutes)
Epoch [8/20], Loss: 0.0599, Train Accuracy: 97.86%, Time: 136.22 seconds (2.27 minutes)
Epoch [9/20], Loss: 0.0329, Train Accuracy: 98.93%, Time: 135.33 seconds (2.26 minutes)
Epoch [10/20], Loss: 0.1863, Train Accuracy: 93.30%, Time: 136.27 seconds (2.27 minutes)
Epoch [11/20], Loss: 0.1245, Train Accuracy: 96.02%, Time: 136.02 seconds (2.27 minutes)
Epoch [12/20], Loss: 0.1773, T

# Evaluate Model on Test Set

In [11]:
batch_size = 32
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [12]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

# Specify the path for loading the paper
model_path = "resnet18_model_bth32_eph20_19.pth"

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
model = torchvision.models.resnet18(pretrained=False)
# Modify the last fully connected layer
model.fc = nn.Linear(model.fc.in_features, 1)

# Load the state dictionary into the model
model.load_state_dict(torch.load(model_path))

# Move the model to the appropriate device
model = model.to(device)

In [13]:
# Set model to evaluation mode
model.eval()

# Initialize variables to store accuracy and loss
test_loss = 0.0
correct_test = 0
total_test = 0

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the loss function
loss_fn = nn.BCEWithLogitsLoss()

# Iterate over the test dataset
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        # Reshape labels for BCEWithLogitsLoss
        labels = labels.float().view(-1, 1)

        # Calculate loss
        loss = loss_fn(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        # Calculate test accuracy
        # Apply sigmoid and round to get predictions
        predicted = torch.round(torch.sigmoid(outputs))
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

# Calculate average loss and accuracy
average_test_loss = test_loss / len(test_set)
test_accuracy = correct_test / total_test
print(f"Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.2%}")

Test Loss: 0.0335, Test Accuracy: 98.44%
